<a href="https://colab.research.google.com/github/EbrahimNabizadeh/TL_Nudge_Losses/blob/master/Creating_a_tf_data_Dataset_from_a_DataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os, requests, copy, random
import matplotlib.pyplot as plt
from tensorflow.data import Dataset

In [2]:
import tensorflow as tf

In [3]:
class ArrayDataset:
    def __init__(self, phase, array, labels, mean, std, transformations = None):
        #super().__init__()
        self.phase = phase
        self.imgarr = copy.deepcopy(array)
        self.labels = copy.deepcopy(labels)
        #self.normalize = transforms.Normalize(mean, std)
        self.mean = mean
        self.std = std
        self.transforms = transformations

    def __len__(self):
        return self.imgarr.shape[0]

    def __getitem__(self,idx):
        x = self.imgarr[idx]
        x = self.augment(x)
        y = self.labels[idx].astype(np.int64)
        return x, y
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def normalize(self, x):
        return (x-self.mean)/self.std
            
    #shuffles the dataset at the end of each epoch
    def on_epoch_end(self):
        reidx = random.sample(population = list(range(self.__len__())),k = self.__len__())
        self.imgarr = self.imgarr[reidx]
        self.labels = self.labels[reidx]
    #applies randomly selected augmentations to each clip (same for each frame in the clip)
    def augment(self, x):
        if self.transforms is not None:
            x = self.transforms(x)
        x = self.normalize(x)
        return x

In [4]:
array = np.random.random((1000,32,32,3))
labels = np.random.randint(0,10,1000)
print(array.shape, labels.shape)
mean = np.mean(array, axis = 0)
std = np.std(array, axis = 0)

(1000, 32, 32, 3) (1000, 32, 32, 3)


In [5]:
dg = ArrayDataset('train',array,labels,mean,std)

In [6]:
ds = Dataset.from_generator(dg, output_types = (tf.float32, tf.int64), output_shapes = (tf.TensorShape([32,32,3]),tf.TensorShape([])))

In [7]:
ds = ds.batch(32)

In [8]:
for b in ds:
    imgs, labs = b
    fig, axs = plt.subplots(8,4)
    for i in range(8):
        for j in range(4):
            axs[i,j].imshow(mean + imgs[i*4+j]*std,cmap = 'gray')
    plt.show()

InvalidArgumentError: ignored